# Microblaze Compilation

This notebook introduces how to compile Microblaze code from within Jupyter and IPython. The examples presented here use the base overlay and the IOPs but this process can be extended to other Microblaze systems.

The first thing we need is to load the Microblaze library.

In [1]:
import ipython_microblaze as ipmb

This package will install a new `%%microblaze_functions` magic which takes the IOP or Microblaze subsystem to run on as an argument. Any C functions in the cell will then be exposed as Python functions that can be called. Input and output to microblaze can be done using the `stdio` attribute of the exported function. Note that all functions will share the same input and output pipe.

## Example 1: Lower-casing some text

This example gives the simplest implementation of reading and writing to a stream inside of the C code and how that is exposed in Python. It reads a characters from `stdin`, lower-cases it all and then writes them back again. First we need to import the Base overlay to access the IOPs

In [2]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

Then we can use one of our new magic cells to compile the program and load it on to the IOP. The first argument is the Microblaze to use and the second parameter is the variable to store the compiled program in.

In [3]:
%%microblaze_functions base.PMODB

void lower_case() {
        char data;
        while (1) {
            read(STDIN_FILENO, &data, 1);
            data = (char)tolower(data);
            write(STDOUT_FILENO, &data, 1);
        }
}

We can now call the function to start the process and communicate using `stdio`. Note that the read and write functions on the input and output streams take binary strings so the Unicode strings will need to be `encode`d or `decode`d.

In [4]:
import time

lower_case()

test_string = 'HELLO, WORLD!'.encode()
lower_case.stdio.write(test_string)
time.sleep(1)
print(lower_case.stdio.read().decode())

hello, world!


## Example 2: `printf` and its variants

This example looks at some of the ways of handling and printing strings inside the microblaze code. The most common function is `printf` which, due to its design, is too large to fit in the microblaze code memory. Instead there are a variety of functions that offer restricted subsets of printf. The full list is detailed a [https://www.xilinx.com/support/answers/19592.html]. The two of most use here are `print` which prints a plain string and `xil_printf` which offers non-reentrant printing to stdout without support for floating point numbers. `xil_io.h` should be included for the `xil_printf` and `print` function prototypes.

The code below prints a header before starting with `print` and then reads a line of text, character by character, from stdin using `getchar`, echoing the characters with `putchar`. Once a whole line has been read the number of characters in the line is written using `xil_printf`.

In [5]:
%%microblaze_functions base.ARDUINO
#include <stdio.h>
#include <xil_io.h>

void letter_count() {
    print("Starting Letter Count\n");
    while (1) {
        int letter_count = 0;
        int c = getchar();
        while (c != '\n') {
            putchar(c);
            c = getchar();
            letter_count++;
        }
        fflush(stdout);
        xil_printf(" (%d letters)\n", letter_count);
    }

}

In [6]:
import time

letter_count()

test_string = "Hello, World!\nA really really long string\n"
letter_count.stdio.write(test_string.encode())

time.sleep(0.2)
print(letter_count.stdio.read().decode())

Starting Letter Count
Hello, World! (13 letters)
A really really long string (27 letters)



## Example 3: Asynchronous Communication
The IOPs have interrupt support on reading so the PS can be idle while waiting for data. The use of `asyncio` allows us to chain the IOPs together using coroutines. This example will feed the result of the lower-case IOP into the input of the counting IOP.

As the whole process will happen asynchronously, we use three tasks to feed the data, transfer the data, and print the result. To ensure that interrupts rather than polling is being used we also have a fourth task to print the CPU utilisation periodically throughout the program's execution.

In [7]:
import asyncio
import psutil

async def write_data():
    for i in range(20):
        lower_case.stdio.write(f"TeSt String {i}\n".encode())
        await asyncio.sleep(0.5)
        
async def transfer_data():
    while True:
        data = await lower_case.stdio.read_async()
        letter_count.stdio.write(data)

async def read_print():
    while True:
        data = await letter_count.stdio.read_async()
        print(data.decode().strip('\n'))

async def print_cpu_usage():
    # Calculate the CPU utilisation by the amount of idle time
    # each CPU has had in three second intervals
    last_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
    while True:
        await asyncio.sleep(3)
        next_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
        usage = [(1-(c2-c1)/3) * 100 for c1,c2 in zip(last_idle, next_idle)]
        print("CPU Usage: {0:3.2f}%, {1:3.2f}%".format(*usage))
        last_idle = next_idle


        
write_task = asyncio.ensure_future(write_data())
transfer_task = asyncio.ensure_future(transfer_data())
read_task = asyncio.ensure_future(read_print())
usage_task = asyncio.ensure_future(print_cpu_usage())

Finally we can run the event loop until the writing task has finished

In [8]:
loop = asyncio.get_event_loop()
loop.run_until_complete(write_task)

test string 0 (13 letters)
test string 1 (13 letters)
test string 2 (13 letters)
test string 3 (13 letters)
test string 4 (13 letters)
test string 5 (13 letters)
CPU Usage: 5.33%, 2.00%
test string 6 (13 letters)
test string 7 (13 letters)
test string 8 (13 letters)
test string 9 (13 letters)
test string 10 (14 letters)
test string 11 (14 letters)
CPU Usage: 3.33%, 2.67%
test string 12 (14 letters)
test string 13 (14 letters)
test string 14 (14 letters)
test string 15 (14 letters)
test string 16 (14 letters)
test string 17 (14 letters)
CPU Usage: 9.00%, 7.33%
test string 18 (14 letters)
test string 19 (14 letters)


To clean up we need to cancel our never-ending tasks to avoid polluting the event loop

In [9]:
transfer_task.cancel()
read_task.cancel()
usage_task.cancel()

lower_case.reset()
letter_count.reset()

## Example 4 - Direct Communication between IOPs

As the lowl-leve stream implementation is symmetric (with the exception of the interrupts which are special-cased), it is entirely possible for two IOPs to talk to each other without needing the PS to do anything. The microblaze runtime implements the `open` syscall where the path is the pointer to the base address of the buffer. First we'll recreate our upper-case/counting demo but with the IOP communication done without the PS.

First we need to allocate some memory for the buffer using the xlnk driver.

In [10]:
from pynq import Xlnk

xlnk = Xlnk()
buffer = xlnk.cma_array(shape=0x800, dtype='u1')

We are going to pass this pointer into each program as an argument to each function. Using a `void*` pointer means the the physical address of the buffer will be passed to the microblaze.

In [11]:
%%microblaze_functions base.PMODA
#include <fcntl.h>
#include <stdio.h>

void letter_count(void* buffer) {
    print("Starting Letter Count\n");
    char* stream_ptr = buffer;
    int fd;
    fd = open(stream_ptr, O_RDONLY);
    FILE* f = fdopen(fd, "r");
    xil_printf("Using descriptor %d on buffer %d\n", fd, (int)stream_ptr);
    while (1) {
        int letter_count = 0;
        int c;
        c = getc(f);
        while (c != '\n') {
            putchar(c);
            c = getc(f);
            letter_count++;
        }
        fflush(stdout);
        xil_printf(" (%d letters)\n", letter_count);
    }

}

In [12]:
%%microblaze_functions base.PMODB
#include <fcntl.h>
#include <stdio.h>

void lower_case(void* buffer_ptr) {
        char* stream_ptr = buffer_ptr;
        char data;
        int fd;
        fd = open(buffer_ptr, O_WRONLY);
        xil_printf("Using descriptor %d on buffer %d\n", fd, (int)buffer_ptr);
        while (1) {
            read(STDIN_FILENO, &data, 1);
            data = (char)tolower(data);
            write(fd, &data, 1);
        }
}

Then we can start our two programs on the same buffer.

In [13]:
letter_count(buffer)
lower_case(buffer)

We can then base our string into the lower microblaze and read the result back from the count program without having to do any data transfer in the PS.

In [14]:
lower_case.stdio.write(b'Hello, World\n')
time.sleep(0.2)
print(letter_count.stdio.read().decode())

Starting Letter Count
Using descriptor 4 on buffer 914644992
hello, world (12 letters)



We can also read the debug information being printed from the lower program as well.

In [15]:
print(lower_case.stdio.read())

b'Using descriptor 4 on buffer 914644992\n'


Unlike other microblaze programs, any which touch the DDR memory need to be closed properly to otherwise invalid transactions can occur when the bitstream is reprogrammed leading to the PS interconnect in an ill-defined state and many hours spent debugging. We can also take this opportunity to free the Xlnk buffer.

In [16]:
lower_case.reset()
letter_count.reset()
buffer.close()

## Example 5 - Analog and Digital I/O

The I/O library is divided into parts - a wrapper around device drivers usable with any microblaze system and an I/O switch-based library for the PMOD and Arduino IOPs which configures the switch based on what is requested. The common library lives in `mbio.h` and contains a typedef for each type of peripheral supported along with C functions for interacting with it. The IO switch library lives in `iop.h` and provides a set of methods for connecting peripherals to specific output pins. The result is a handle which can then be used with the common library to actually perform the I/O.

As an example lets create an example which flashes an LED and uses the following APIs

```C
gpio gpio_open_iop_grove(unsigned char port, unsigned char wire);

void gpio_set_direction(gpio dev, int direction);
void gpio_write(gpio dev, int val);
```

In [17]:
%%microblaze_functions base.PMODA
#include <iop.h>

void led_flash() {
    // Connect to an LED attached to the first pin on G2
    gpio led = gpio_open_iop_grove(G2, 0);
    gpio_set_direction(led, GPIO_OUT);
    int state = 0;
    while (1) {
        gpio_write(led, state);
        delay(500);
        state = !state;
    }
}

We can now start the flashing program.

In [18]:
led_flash()

And reset the microblaze to stop the flashing and prepare for another program to be executed.

In [19]:
led_flash.reset()

Analog I/O is currently specific to the Arduino header which has six analog inputs split across 4 grove connectors in the standard shield. The analog API looks very similar to the GPIO API. There are additional methods which can be used to read the raw value explained in the Analog and GPIO notebook.

```C
analog analog_open_iop_grove(unsigned char port, unsigned char wire);

float analog_read(analog device);
```

In [20]:
%%microblaze_functions base.ARDUINO
#include <iop.h>

void read_adc(float* values, int number) {
    analog device = analog_open_iop_grove(A1, 0);
    for (int i = 0; i < number; ++i) {
        float data = analog_read(device);
        values[i] = data;
        delay(1000);
    }
}

We can now read 10 values from the ADC.

In [21]:
result = [0.0] * 10
read_adc(result, 10)
print(result)

[3.271096706390381, 3.2713987827301025, 3.2707443237304688, 3.2213973999023438, 2.2173385620117188, 1.7152587175369263, 1.438513159751892, 1.1690185070037842, 0.9079330563545227, 0.6375823616981506]


In [22]:
read_adc.reset()

## Example 6 - Modules

To support more complex devices, the `ipythonmicroblaze` package uses _modules_ to extend the base functionality. A couple of example modules are included in this package but the plan is that many more are made available as additional packages.

All available modules are listed in the `Modules` dict.

In [23]:
ipmb.Modules.keys()

dict_keys(['pmod_oled', 'ledbar'])

Each modules consists of at least a header file which can be retrieved from python to expose the API. Including the header in your project will automatically pull in the correct sources and libraries.

In [24]:
print(ipmb.Modules['ledbar'].header)

#pragma once

#include <mbio.h>

typedef unsigned int ledbar;

ledbar ledbar_open_grove(unsigned char port);
ledbar ledbar_open(gpio data, gpio clk);
void ledbar_set_level(ledbar, unsigned char i);
void ledbar_set_data(ledbar, const unsigned char* data);




We can now use the `ledbar` module in one of our example programs. By convention the main API header is named the same as the module name.

In [25]:
%%microblaze_functions base.ARDUINO
#include <ledbar.h>
#include <iop.h>

void set_ledbar(const unsigned char* data) {
    ledbar g1 = ledbar_open_grove(G1);
    ledbar_set_data(g1, data);
}

We can now set the LED bar to show an alternating pattern. As we are not using a `void*` pointer the data will be packed and copied over.

In [26]:
set_ledbar(b'\xFF\x00' * 5)

In [27]:
set_ledbar.reset()

## Example 7 - Offloaded Computation

Now we have snippets for both our LED bar and ADC/temperature sensor we can combine them into a single program that reads the voltage and displays it on the LED bar. To determine what the levels should be we pass in 10 float values at startup that correspond to the threshold voltages for each LED.

In [28]:
%%microblaze_functions base.ARDUINO
#include <iop.h>
#include <ledbar.h>

void led_adc(const float* thresholds) {
    analog sensor = analog_open_iop_grove(A1, 0);
    ledbar output = ledbar_open_grove(G1);

    while (1) {
        float voltage = analog_read(sensor);
        int val = 0;
        for (int i = 0; i < 10; ++i) {
            if (voltage > thresholds[i]) val = i + 1;
        }
        ledbar_set_level(output, val);
    }
}

In [29]:
led_adc([0.3 + 0.3 * i for i in range(10)])

In [30]:
led_adc.reset()